In [49]:
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* **`19`**

In [33]:
!mkdir -p data
!wget -qq -c https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O data/yellow_tripdata_2023-01.parquet
!wget -qq -c https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -O data/yellow_tripdata_2023-02.parquet
print("Download done")

Download done


In [34]:
df1 = pd.read_parquet("data/yellow_tripdata_2023-01.parquet", engine="pyarrow")

In [35]:
df1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [36]:
print(f"#columns: {len(df1.columns)}")

#columns: 19


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* **`42.59`**
* 52.59
* 62.59


In [37]:
df1["tpep_pickup_datetime"] = pd.to_datetime(df1["tpep_pickup_datetime"])
df1["tpep_dropoff_datetime"] = pd.to_datetime(df1["tpep_dropoff_datetime"])
df1["duration"] = df1["tpep_dropoff_datetime"] - df1["tpep_pickup_datetime"]
df1["duration"] = df1["duration"].dt.total_seconds() / 60

print(f"std(duration): {df1['duration'].std():.2f}")

std(duration): 42.59


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* **`98%`**

In [38]:
rows_old = df1.shape[0]
df1 = df1[(df1["duration"] >= 1) & (df1["duration"] <= 60)].copy()
rows = df1.shape[0]

fraction = (rows / rows_old) * 100
print(f"Fraction of remaining records: {fraction:.0f}%")

Fraction of remaining records: 98%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* **`515`**
* 715

In [40]:
categorical = ["PULocationID", "DOLocationID"]
df1[categorical] = df1[categorical].astype(str)
train_dicts = df1[categorical].to_dict(orient="records")
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [43]:
print(f"#columns: {X_train.shape[1]}")

#columns: 515


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* **`7.64`**
* 11.64
* 16.64

In [65]:
target = "duration"
y_train = df1[target]

In [66]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [67]:
y_pred = model.predict(X_train)
print(f"RMSE(Train): {mean_squared_error(y_train, y_pred, squared=False):.2f}")

RMSE(Train): 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* **`7.81`**
* 11.81
* 16.81

In [72]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].astype('str')
    
    return df

In [73]:
# Read validation data (data from feb 2023)
df_val = read_data("data/yellow_tripdata_2023-02.parquet")

In [74]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values
y_pred = model.predict(X_val)

In [76]:
print(f'Val RMSE: {mean_squared_error(y_val, y_pred, squared=False):.2f}')

Val RMSE: 7.81
